# Rumana Khan and Peter Hughes LC

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///lab3.db3

'Connected: @lab3.db3'

In [3]:
%sql select * from Scores;

 * sqlite:///lab3.db3
Done.


sno,cno,grade
103,3-245,86
105,3-245,75
109,3-245,68
103,3-105,92
105,3-105,88
109,3-105,76
101,3-105,64
107,3-105,91
108,3-105,78
101,6-166,85


In [4]:
%sql select * from Students;

 * sqlite:///lab3.db3
Done.


sno,sname,sgender,sbirthday,class
108,Peter,M,1997-09-01,15033
105,Gary,M,1995-10-02,15031
107,Lily,F,1996-01-23,15033
101,Alex,M,1996-02-20,15033
109,Eva,F,1995-02-10,15031
103,Quincy,M,1994-06-03,15031


In [5]:
%sql select * from Courses;

 * sqlite:///lab3.db3
Done.


cno,cname,tno
3-105,Introduction to Computer Science,825
3-245,Operating Systems,804
6-166,Digital Circuit,856
9-888,Advanced Mathematics,100


In [6]:
%sql select * from Teachers;

 * sqlite:///lab3.db3
Done.


tno,tname,tgender,tbirthday,title,department
804,Dan,M,1958-12-02,AP,CS
856,Frank,M,1969-03-12,LE,EE
825,Alice,F,1972-05-05,TA,CS
831,Emma,F,1977-08-14,TA,EE


Q1-- We used a nested query here to give back the results that has grades that were in the range we were given (70-90). After that we used the group by and having count function to make sure that only students who had received all of their scores in that range were being returned the query. We set the count at 2 because clearly each student took 2 courses; thus, they only had 2 grades in the Scores table. 

In [16]:
%%sql 
SELECT B.sno
from Students B, Scores S
WHERE S.grade IN (
    SELECT grade 
    from Scores 
    Where grade > 69 and grade < 91)
AND S.sno = B.sno
GROUP BY S.sno
HAVING COUNT(*) = 2

 * sqlite:///lab3.db3
Done.


sno
105
108


Q2-- This question was very straight foward. We just return the overall average grade that the class received since it said "for all the courses." 

In [8]:
%%sql 
SELECT AVG(grade)
from Scores, Students
WHERE Students.class = '15033'
and Scores.sno = Students.sno

 * sqlite:///lab3.db3
Done.


AVG(grade)
79.66666666666667


Q3-- For the third question, we used again a nested query where the second query checks that the count(or number) of males in the class is equal to or higher than 2.

In [14]:
%%sql 
SELECT DISTINCT B.class
from Students B
WHERE EXISTS (SELECT COUNT(sgender) >= 2 from Students where sgender = 'M')

 * sqlite:///lab3.db3
Done.


class
15033
15031


Q4-- For this question, we simply made sure that the department and title for the teachers being compared were not the same.  We also used group-by to organize the information and to remove any duplicate pairs. 

In [19]:
%%sql
SELECT D.title, D.tname
from Teachers D, Teachers S
WHERE D.department != S.department and D.title != S.title
GROUP BY D.tname

 * sqlite:///lab3.db3
Done.


title,tname
TA,Alice
AP,Dan
TA,Emma
LE,Frank


Q5-- We used the Scores table twice here and we set one of them as class 3-105 and the other as 3-245. Then we made sure to compare the grades and only return those that were higher than the lowest grade in 3-245. Finally, we simply ordered the grades in 3-105 in a descending order using order by.

In [6]:
%%sql
SELECT DISTINCT A.sname
from Students A, Scores S, Scores T
WHERE A.sno = S.sno and S.cno ='3-105'
    and T.cno = '3-245' and S.grade > T.grade
ORDER BY S.grade DESC

 * sqlite:///lab3.db3
Done.


sname
Quincy
Lily
Gary
Peter
Eva


Q6-- First, we use a nested query to find students who took more than one course by using the Scores Table twice and making sure the course numbers do not match each other, which clearly meant that the student was in more than one course. Then we used except to take out the highest student buy utilizing the MAX function to find the highest grade a student scored. 

In [7]:
%%sql 
SELECT distinct sname
from Scores, Students
where Scores.sno = Students.sno 
and Students.sno in (select A.sno
                     from Scores A, Scores B
                     where A.cno <> B.cno)
Except 
SELECT distinct B.sname
from Scores A, Students B
where (SELECT MAX(grade) = A.grade from Scores) and A.sno = B.sno

 * sqlite:///lab3.db3
Done.


sname
Alex
Eva
Gary
Lily
Peter


Q7--Finally, the last question! We used a nested query to return instances where the avergae grade was higher than the students grade. Then we returned that student's student number and name. We ordered the information with Course number. It is ascending order. First, 101,108,109 is for 3-105. Then, 105 and 109 for 3-245. Finally, 107 for 6-166.

In [15]:
%%sql
SELECT A.sname, A.sno
from Students A, Scores B
where (SELECT AVG(grade) > B.grade from Scores) and A.sno = B.sno
ORDER BY B.cno

 * sqlite:///lab3.db3
Done.


sname,sno
Alex,101
Peter,108
Eva,109
Gary,105
Eva,109
Lily,107
